# Creation of a digital twin 
This is a KWH application which allows creating a simple digital twin using python reference implementation. The created digital twin can be launched and connect to S³I in this notebook, until you shut down the kernel. In this notebook, we will borrow the example of harvester to show you how to create a digital twin. All necessary modules are imported into the script, including the S³I library and reference implementation library. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [1]:
import s3i
import jwt
import requests
import getpass
import json 
from ml.tools import load_config, make_thing_config, make_sub_thing
from ml.dt_factory import create_dt_ref
from ml.app_logger import APP_LOGGER, setup_logger
from tools import print_with_timestamp, check_message_encryption, yes, no, dt_creation_app_id, dt_creation_app_secret

## Configure the notebook
In order to connect to S³I this notebook needs a client id and the respective secret to authenticate itself. The client credentials are already prepared for you. You only need to enter your username and password in the following input fields, so that you can assign this notebook to your private KWH application. 

In [2]:
print_with_timestamp("KWH application to create a dt, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=dt_creation_app_id,
                                             client_secret=dt_creation_app_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

[S3I][2020-10-09 07:47:40]: KWH application to create a dt, please log in!
[S3I]: Please enter your username:chen
[S3I]: Please enter the password:········
[S3I][2020-10-09 07:47:47]: Your credentials are sent to S3I IdentityProvider.
[S3I][2020-10-09 07:47:47]: Token received, chen logged in.


## Create an identity for your digital twin
Your digital twin needs an identify to connect with S³I, which can be created using S³I config rest api. 

In [3]:
s3i_config = s3i.Config(access_token)
resp = s3i_config.create_thing()
dt_id = resp.json().get("identifier", None)
dt_secret = resp.json().get("secret", None)
s3i_config.create_broker_queue(thing_id=dt_id)
print_with_timestamp("Your digital twin has the identifier: {}".format(dt_id))

[S3I][2020-10-09 07:47:48]: Your digital twin has the identifier: s3i:461feada-29d8-4d92-be4e-cabda5fb857e


## Configurate your digital twin of harvester
as next we will introduce how to simply configurate your digital twin of harvester using python reference implementation library. All the hereby used roles and features have been defined in fml40 and implemented in this reference implementation library. 

In [4]:
dt_name = input("[S³I]: Please name your digital twin Harvester(e.g. my_harvester): ")
setup_logger(dt_name)
config_engine = make_sub_thing(name="my_engine", roles=[{"class": "ml40::Engine"}],
                               features=[ {"class": "ml40::RotationalSpeed", "rpm": 5}])

config_cran = make_sub_thing(name="my_bord_computer", roles=[{"class": "ml40::MachineUI"}])
config_file_name = make_thing_config(dt_id=dt_id, name=dt_name, roles=[{"class": "fml40::Harvester"}],
                                     features=[{"class": "fml40::ProvidesProductionData"},
                                              {"class": "fml40::AcceptsFellingJobs"},
                                              {"class": "ml40::Composite",
                                               "targets": [config_engine, config_cran]}])

dt_model = load_config('configs/{}'.format(config_file_name))
print_with_timestamp("The date model of the created harvester: " + json.dumps(dt_model, indent=2))

[S³I]: Please name your digital twin Harvester(e.g. my_harvester): my_harvester
[S3I][2020-10-09 07:47:52]: The date model of the created harvester: {
  "thingId": "s3i:461feada-29d8-4d92-be4e-cabda5fb857e",
  "policyId": "s3i:461feada-29d8-4d92-be4e-cabda5fb857e",
  "attributes": {
    "class": "ml40::Thing",
    "name": "my_harvester",
    "roles": [
      {
        "class": "fml40::Harvester"
      }
    ],
    "features": [
      {
        "class": "fml40::ProvidesProductionData"
      },
      {
        "class": "fml40::AcceptsFellingJobs"
      },
      {
        "class": "ml40::Composite",
        "targets": [
          {
            "class": "ml40::Thing",
            "name": "my_engine",
            "roles": [
              {
                "class": "ml40::Engine"
              }
            ],
            "features": [
              {
                "class": "ml40::RotationalSpeed",
                "rpm": 5
              }
            ]
          },
          {
            

## Create the reference and proxy of your digital twin Harvester
With the configuration file you can in the next steps create harvester reference and proxy. The digital twin will register the features and roles and bind himself with those structurally. 

In [5]:
dt_ref = create_dt_ref(model=dt_model, grant_type="password", secret=dt_secret, username=username, password=password,
                       is_broker_rest=True,
                       is_broker=True, is_repo=False)
dt_proxy = dt_ref.proxy()

2020-10-09 07:47:52,159 - DEBUG - my_harvester: Creating ditigtal twin my_harvester with id s3i:461feada-29d8-4d92-be4e-cabda5fb857e
DEBUG:app_logger:Creating ditigtal twin my_harvester with id s3i:461feada-29d8-4d92-be4e-cabda5fb857e
2020-10-09 07:47:52,165 - DEBUG - my_harvester: Adding roles: fml40::Harvester
DEBUG:app_logger:Adding roles: fml40::Harvester
2020-10-09 07:47:52,169 - DEBUG - my_harvester: Adding feature: fml40::ProvidesProductionData
DEBUG:app_logger:Adding feature: fml40::ProvidesProductionData
2020-10-09 07:47:52,174 - DEBUG - my_harvester: Adding feature: fml40::AcceptsFellingJobs
DEBUG:app_logger:Adding feature: fml40::AcceptsFellingJobs
2020-10-09 07:47:52,178 - DEBUG - my_harvester: Adding feature: ml40::Composite
DEBUG:app_logger:Adding feature: ml40::Composite
2020-10-09 07:47:52,184 - DEBUG - my_harvester: Creating ditigtal twin my_engine with id 
DEBUG:app_logger:Creating ditigtal twin my_engine with id 
2020-10-09 07:47:52,186 - DEBUG - my_harvester: Adding

## Run the digital twin Harvester
The created digital twin of harvester will be launched and connect directly to S³I, until you manuelly shutdown the kernel of this notebook. Additionally, his registered services will response incoming service requests. 

In [6]:
dt_proxy.run_forever()

[S³I]: Launch my_harvester
[S³I][IdP]: Connect with S3I-IdentityProvider
[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker


2020-10-09 07:49:26,477 - INFO - my_harvester: Checking if the felling job can be accepted.
INFO:app_logger:Checking if the felling job can be accepted.
2020-10-09 07:49:26,479 - INFO - my_harvester: Job accepted!
INFO:app_logger:Job accepted!


[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker
[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker
